In [222]:
import pickle
import pandas as pd
import numpy as np
from ordered_set import OrderedSet

#### 우선, author dictionary 는 해당 페이지에서 작업하지 않는다. 
#### 해당 파일에서 하려는 것은 딕셔너리의 성능 향샹을 위해서, 데이터프레임으로 딕셔너리를 효과적으로 시각화(visualize), 
#### 자의적으로 두 개 이상(N) 으로 나뉘어진 딕셔러니의 key들을 N-1 이하 개의 key 로 줄여주기 위함이다.

In [223]:
pickle_in = open("dictionary/paper_ready_to_target_dict.pickle","rb")
paper_ready_to_target_dict = pickle.load(pickle_in)

In [224]:
type(paper_ready_to_target_dict)

(dict, dict)

In [225]:
paper_dict = paper_ready_to_target_dict.copy()

for key in paper_dict.keys() : 
    paper_dict[key] = set(paper_dict.get(key))

In [226]:
paper_dict_df = pd.DataFrame.from_dict(paper_dict,orient='index')

In [227]:
paper_dict_df['year'] = [i.split("////")[1] for i in paper_dict.keys()]

In [228]:
paper_dict_df.shape , author_dict_df.shape

((853, 14), (1061, 10))

In [229]:
paper_split_ls = [i.split('////') for i in paper_dict_df.index.tolist()]
paper_split_ls[:5]

[['on persistence in mutual fund performance', '1997'],
 ['the pricing of options and corporate liabilities', '1973'],
 ['tunneling', '2000'],
 ['corporate financing and investment decisions when firms have information that investors do not have',
  '1984'],
 ['the cross-section of volatility and expected returns', '2006']]

딕셔너리의 key들끼리 similarity를 계산한다. 눈으로 보고 병합할 것을 판단한다.

In [230]:
def calc_similarity_score(t1,t2) : 
    t1 = OrderedSet(t1[0].split(" "))
    t2 = OrderedSet(t2[0].split(" "))
    score = np.average([len(t1.intersection(t2)) / len(t1),len(t1.intersection(t2)) / len(t2)])
    return score

In [231]:
testing_ls = paper_split_ls.copy()

for idx1 in range(len(testing_ls)) : 
    for idx2 in range(len(testing_ls)) : 
        if idx1 >= idx2 : continue
        try : 
            if np.abs(int([i[1] for i in testing_ls][idx1]) - int([i[1] for i in testing_ls][idx2])) < 2 : 
                if calc_similarity_score(testing_ls[idx1],testing_ls[idx2]) > 0.75 : 
                    print((testing_ls[idx1],testing_ls[idx2]))
        except : 
            if calc_similarity_score(testing_ls[idx1],testing_ls[idx2]) > 0.75 : 
                print((testing_ls[idx1],testing_ls[idx2]))


(['the general theory of employment interest and money', '1936'], ['the general theory of employment', '1937'])
(['the maturity structure of corporate debt', '1995'], ['the determinants of corporate debt maturity structure', '1996'])


`['the general theory of employment interest and money', '1936'], ['the general theory of employment', '1937']` 이거 두 개는 병합해야 한다.

In [232]:
paper_dict_df.sort_values('year',inplace=True)
paper_dict_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,year
the modern corporation and private property////1932,the modern corporation and private property///...,None,None,None,None,None,None,None,None,None,None,None,None,1932
interest and money////1936,interest and money////1936,None,None,None,None,None,None,None,None,None,None,None,None,1936
the general theory of employment interest and money////1936,the general theory of employment. in: interest...,the general theory of employment interest and ...,interest and money////1936,None,None,None,None,None,None,None,None,None,None,1936
the general theory of employment////1937,the general theory of employment////1937,None,None,None,None,None,None,None,None,None,None,None,None,1937
individual comparisons by ranking methods////1945,individual comparisons by ranking methods////1945,None,None,None,None,None,None,None,None,None,None,None,None,1945


In [233]:
paper_dict_df.iat[2,3] = paper_dict_df.iat[3,0]

In [235]:
paper_dict_df.drop(["the general theory of employment////1937"],inplace=True)

In [236]:
paper_dict_df.drop(['year'],axis=1,inplace=True)

In [238]:
key_list = paper_dict_df.index.tolist()

In [242]:
paper_dict_df.shape[0]

852

In [243]:
value_list = []

for idx in range(paper_dict_df.shape[0]) : 
    value_list.append([i for i in paper_dict_df.iloc[idx,:].tolist() if i])

In [247]:
new_paper_dict = dict(zip(key_list,value_list))

In [255]:
for key in list(new_paper_dict.keys()) : 
    if len(new_paper_dict.get(key)) < 1: 
        del new_paper_dict[key]
        print('delete!')

delete!
delete!
delete!
delete!
delete!


In [257]:
pickle_out = open("new_paper_ready_to_target_dict.pickle","wb")
pickle.dump(new_paper_dict, pickle_out)
pickle_out.close()